텍스트 데이터 다루기
---

텍스트 데이터는 전처리를 잘 해야 한다.

**로컬에서 실행할 때 소요되는 시간이 지나치게 길어서 코드 및 내용만 적는 것으로 학습하였음**

### 7.1 문자열 데이터 타입

자주 등장하는 유형 정리
- 범주형 데이터 : 고정된 목록으록 구성(빨, 주, 노, 파...등등)
- 범주형 의미를 연결시킬 수 있는 임의의 문자열 : 포용 가능한 범주를 정의해서 매핑
- 구조화된 문자열 데이터 : 주소, 장소, 사람 이름 등 처리하기 어려운 것(이 책에서는 안 다룸)
- 텍스트 데이터
  - 데이터셋 : 말뭉치 corpus
  - 하나의 텍스트를 의미하는 각 데이터 포인트를 문서라고 함 documents

### 7.2 예제! 영화 리뷰 감성 분석
- 영화 리뷰 데이터셋 imdb를 이용해서 분석해보자
- ~~회사에서 데이터셋 받는데 세월이다...~~

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_files

In [3]:
reviews_train = load_files('aclImdb/train/')

In [4]:
text_train, y_train = reviews_train.data, reviews_train.target
print("text_train의 타입:", type(text_train))
print("text_train의 길이:", len(text_train))
print("text_train[6]:\n", text_train[6])

text_train의 타입: <class 'list'>
text_train의 길이: 75000
text_train[6]:
 b'Gloomy Sunday - Ein Lied von Liebe und Tod directed by Rolf Sch\xc3\xbcbel in 1999 is a romantic, absorbing, beautiful, and heartbreaking movie. It started like Jules and Jim; it ended as one of Agatha Christie\'s books, and in between it said something about love, friendship, devotion, jealousy, war, Holocaust, dignity, and betrayal, and it did better than The Black Book which is much more popular. It is not perfect, and it made me, a cynic, wonder in the end on the complexity of the relationships and sensational revelations, and who is who to whom but the movie simply overwhelmed me. Perfect or not, it is unforgettable. All four actors as the parts of the tragic not even a triangle but a rectangle were terrific. I do believe that three men could fell deeply for one girl as beautiful and dignified as Ilona in a star-making performance by young Hungarian actress Erica Marozs\xc3\xa1n and who would not? The titular s

In [5]:
# /br 문자열 정리

text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

In [6]:
print("클래스별 샘플 수 (훈련 데이터):", np.bincount(y_train))

클래스별 샘플 수 (훈련 데이터): [12500 12500 50000]


In [7]:
reviews_test = load_files("aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
print("테스트 데이터의 문서 수:", len(text_test))
print("클래스별 샘플 수 (테스트 데이터):", np.bincount(y_test))
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

테스트 데이터의 문서 수: 25000
클래스별 샘플 수 (테스트 데이터): [12500 12500]


**우리가 풀려고 하는 문제**
- 리뷰가 하나 주어졌을 때, 이 리뷰의 텍스트 내용을 보고 '양성'인지 '음성'인지 구분하는 것
- 전형적인 이진 분류 문제
- 텍스트 데이터는 머신러닝 모델이 다룰 수 있는 형태가 아님
- 그래서 텍스트의 문자열 표현을 머신러닝 알고리즘에 적용할 수 있도록 수치 표현으로 바꿔야 함

### 7.3 텍스트 데이터를 BOW로 표현하기

**BOW**
- bag of words
- 장, 문단, 문장, 서식 같은 입력 텍스트의 구조 대부분을 잃고, 각 단어가 이 말뭉치에 있는 텍스트에 얼마나 많이 나타나는지만 헤아림
- 텍스트를 담는 가방

전체 말뭉치에 대해 BOW 표현을 계산하려면 다음 세 단계를 거쳐야 함
- Tokenization : 토큰화, 각 문서를 문서에 포함된 단어(토큰)로 나눕니다. 예를 들어 공백이나 구두점 등을 기준으로 분리합니다.
- 어휘 사전 구축 : 모든 문서에 나타난 모든 단어의 어휘를 모으고 번호를 매깁니다.(알파벳 순서 등)
- 인코딩 : 어휘 사전의 단어가 문서마다 몇 번이나 나타나는지를 헤아립니다.

#### 7.3.1 샘플 데이터에 BOW 적용하기

In [8]:
bards_words =["The fool doth think he is wise,",
              "but the wise man knows himself to be a fool"]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
vect = CountVectorizer()
vect.fit(bards_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [11]:
print("어휘 사전의 크기:", len(vect.vocabulary_))
print("어휘 사전의 내용:\n", vect.vocabulary_)

어휘 사전의 크기: 13
어휘 사전의 내용:
 {'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


In [12]:
bag_of_words = vect.transform(bards_words)
print("BOW:", repr(bag_of_words))

BOW: <2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


BOW 표현은 0이 아닌 값만 저장하는 Scipy 희소 행렬로 저장되어 있다.
- 값이 0인 원소를 모두 저장하는 것음 메모리 낭비
- toarray 메소드를 이용해서 살펴보자

In [13]:
print("BOW의 밀집 표현:\n", bag_of_words.toarray())

BOW의 밀집 표현:
 [[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


#### 7.3.2 영화 리뷰에 대한 BOW

In [14]:
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print("X_train:\n", repr(X_train))

X_train:
 <75000x124255 sparse matrix of type '<class 'numpy.int64'>'
	with 10315542 stored elements in Compressed Sparse Row format>


In [15]:
feature_names = vect.get_feature_names()

print("특성 개수:", len(feature_names))
print("처음 20개 특성:\n", feature_names[:20])
print("20010에서 20030까지 특성:\n", feature_names[20010:20030])
print("매 2000번째 특성:\n", feature_names[::2000])

특성 개수: 124255
처음 20개 특성:
 ['00', '000', '0000', '0000000000000000000000000000000001', '0000000000001', '000000001', '000000003', '00000001', '000001745', '00001', '0001', '00015', '0002', '0007', '00083', '000ft', '000s', '000th', '001', '002']
20010에서 20030까지 특성:
 ['cheapen', 'cheapened', 'cheapening', 'cheapens', 'cheaper', 'cheapest', 'cheapie', 'cheapies', 'cheapjack', 'cheaply', 'cheapness', 'cheapo', 'cheapozoid', 'cheapquels', 'cheapskate', 'cheapskates', 'cheapy', 'chearator', 'cheat', 'cheata']
매 2000번째 특성:
 ['00', '_require_', 'aideed', 'announcement', 'asteroid', 'banquière', 'besieged', 'bollwood', 'btvs', 'carboni', 'chcialbym', 'clotheth', 'consecration', 'cringeful', 'deadness', 'devagan', 'doberman', 'duvall', 'endocrine', 'existent', 'fetiches', 'formatted', 'garard', 'godlie', 'gumshoe', 'heathen', 'honoré', 'immatured', 'interested', 'jewelry', 'kerchner', 'köln', 'leydon', 'lulu', 'mardjono', 'meistersinger', 'misspells', 'mumblecore', 'ngah', 'oedpius', 'overwhelmi

- 희소 행렬의 고차원 데이터셋에서는 LogisticRegression 같은 선형 모델의 성능이 가장 뛰어남

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:
scores = cross_val_score(LogisticRegression(), X_train, y_train, cv=5)
print("크로스 밸리데이션 평균 점수: {:.2f}".format(np.mean(scores)))

/Users/wontaek/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/wontaek/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [17]:
from sklearn.model_selection import GridSearchCV

In [13]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [ ]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("최상의 크로스 밸리데이션 점수: {:.2f}".format(grid.best_score_))
print("최적의 매개변수: ", grid.best_params_)

단어 추출 방법을 개선할 차례
- CountVectorizer는 정규표현식을 사용하여 토큰을 추출함
- 대문자를 소문자로 바꿈
- 그 외는 공식문서 참조 - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
- 의미없는 특성이나 숫자를 줄이는 방법은 적어도 두개의 문서에 나타난 토큰만을 사용하는 것
- 하나의 문서에서만 나타난 토큰은 테스트 세트에 나타날 가능성이 적으므로 그리 큰 도움이 되지 않는다.
- min_df 매개변수로 토큰이 나타날 최소 문서 개수를 지정할 수 있다.

In [10]:
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
print("min_df로 제한한 X_train:", repr(X_train))

min_df로 제한한 X_train: <75000x44532 sparse matrix of type '<class 'numpy.int64'>'
	with 10191240 stored elements in Compressed Sparse Row format>


In [11]:
feature_names = vect.get_feature_names()

print("First 50 features:\n", feature_names[:50])
print("Features 20010 to 20030:\n", feature_names[20010:20030])
print("Every 700th feature:\n", feature_names[::700])

First 50 features:
 ['00', '000', '001', '007', '00am', '00pm', '00s', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '1001', '100k', '100th', '100x', '101', '101st', '102', '103', '104', '105', '106', '107', '108', '109', '10am', '10pm', '10s', '10th', '10x', '11', '110', '1100', '110th', '111', '112', '1138', '115', '116', '117', '11pm', '11th']
Features 20010 to 20030:
 ['inert', 'inertia', 'inescapable', 'inescapably', 'inevitability', 'inevitable', 'inevitably', 'inexcusable', 'inexcusably', 'inexhaustible', 'inexistent', 'inexorable', 'inexorably', 'inexpensive', 'inexperience', 'inexperienced', 'inexplicable', 'inexplicably', 'inexpressive', 'inextricably']
Every 700th feature:
 ['00', 'accountability', 'alienate', 'appetite', 'austen', 'battleground', 'bitten', 'bowel', 'burton', 'cat', 'choreographing', 'collide', 'constipation', 'creatively', 'dashes', 'descended', 'dishing', 'dramatist', 'ejaculation', 'epitomize', 'extinguished', 'figment', 'forg

In [15]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("최적의 크로스 밸리데이션 점수: {:.2f}".format(grid.best_score_))

### 7.4 불용어

불용어를 없애보자!!

In [18]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [19]:
print("불용어 개수:", len(ENGLISH_STOP_WORDS))
print("매 10번째 불용어:\n", list(ENGLISH_STOP_WORDS)[::10])

불용어 개수: 318
매 10번째 불용어:
 ['once', 'whereupon', 'de', 'ten', 'ever', 'otherwise', 'must', 'yours', 'them', 'in', 'even', 'i', 'have', 'why', 'but', 'become', 'whole', 'during', 'somewhere', 'enough', 'per', 'at', 'top', 'already', 'formerly', 'my', 'serious', 'find', 'towards', 'thick', 'became', 'hundred']


In [20]:
# stop_words="english"라고 지정하면 내장된 불용어를 사용합니다.
# 내장된 불용어에 추가할 수도 있고 자신만의 목록을 사용할 수도 있습니다.

vect = CountVectorizer(min_df=5, stop_words="english").fit(text_train)
X_train = vect.transform(text_train)
print("불용어가 제거된 X_train:\n", repr(X_train))

불용어가 제거된 X_train:
 <75000x44223 sparse matrix of type '<class 'numpy.int64'>'
	with 6577418 stored elements in Compressed Sparse Row format>


In [ ]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("최상의 크로스 밸리데이션 점수: {:.2f}".format(grid.best_score_))

/Users/wontaek/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/wontaek/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/wontaek/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(CountVectorizer(), LogisticRegression())
param_grid = {'countvectorizer__max_df': [100, 1000, 10000, 20000], 'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print("최상의 크로스 밸리데이션 점수: {:.2f}".format(grid.best_score_))
print(grid.best_params_)

너무 오래 걸려서 할 수가 없다;;;

### 7.5 tf-idf로 데이터 스케일 변경하기

중요하지 않아 보이는 특성을 제외하는 대신 얼마나 의미 있는 특성인지를 계싼해서 스케일을 조정하는 방식이 있다.
- 가장 널러 알려진 방식은 term frequency - inverse document frequency
- 줄여서 TF-IDF
- 말뭉치의 다른 문서보다 특정 문서에 자주 나타나는 단어에 높은 가중치를 주는 방법

> 한 단어가 특정 문서에 자주 나타나고 다른 여러 문서에서는 그렇지 않다면 그 문서의 내용을 아주 잘 설명하는 단어라고 볼 수 있다.

sklearn에서는 두 개의 파이썬 클래스에 tf-idf를 구현
- TfidfTransformer는 CountVectorizer가 만든 희소 행렬을 입력받아 변환
- TFidfVectorizer는 텍스트 데이터를 입력받아 BOW 특성 추출과 tf-idf 변환을 수행

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

In [ ]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print("최상의 크로스 밸리데이션 점수: {:.2f}".format(grid.best_score_))

tf-idf는 어떤 단어가 가장 중요한지도 알려준다.
- tf-idf 변환은 문서를 구별하는 단어를 찾는 방법이지만 완전히 비지도 학습
- 그래서 우리의 관심사인 긍정적인 리뷰와 부정적인 리뷰 레이블과 꼭 관계있지 않다는 게 중요함

In [ ]:
vectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
# 훈련 데이터셋을 변환합니다
X_train = vectorizer.transform(text_train)
# 특성별로 가장 큰 값을 찾습니다
max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
# 특성 이름을 구합니다
feature_names = np.array(vectorizer.get_feature_names())

print("가장 낮은 tfidf를 가진 특성:\n",
      feature_names[sorted_by_tfidf[:20]])

print("가장 높은 tfidf를 가진 특성: \n",
      feature_names[sorted_by_tfidf[-20:]])

In [ ]:
sorted_by_idf = np.argsort(vectorizer.idf_)
print("가장 낮은 idf를 가진 특성:\n",
       feature_names[sorted_by_idf[:100]])

In [26]:
import mglearn

/opt/conda/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [29]:
mglearn.tools.visualize_coefficients(
    grid.best_estimator_.named_steps["logisticregression"].coef_[0],
    feature_names, n_top_features=40)

### 7.7 여러 단어로 만든 BOW(n-그램)

BOW 표현 방식은 단어의 순서가 완전히 무시된다는 큰 단점이 존재
- 문맥을 고려하는 방법은 토큰 하나의 횟수만 고려하지 않고 옆에 있는 두세 개의 토큰을 함께 고려하는 방식
- 토큰 두개를 bigram
- 토큰 세 개를 trigram
- 일반적으로 연속된 토큰을 n-gram

In [30]:
print("bards_words:\n", bards_words)

bards_words:
 ['The fool doth think he is wise,', 'but the wise man knows himself to be a fool']


In [32]:
# 기본값은 최소 길이가 1이고 최대 길이가 1인 토큰마다 하나의 특성을 만듬
# 토큰 하나를 유니그램(unigram)이라고 함

cv = CountVectorizer(ngram_range=(1, 1)).fit(bards_words)
print("어휘 사전 크기:", len(cv.vocabulary_))
print("어휘 사전:\n", cv.get_feature_names())

어휘 사전 크기: 13
어휘 사전:
 ['be', 'but', 'doth', 'fool', 'he', 'himself', 'is', 'knows', 'man', 'the', 'think', 'to', 'wise']


In [33]:
# 토큰 2개가 연속된 바이그램만 만드려면 아래와 같이 하면 된다.

cv = CountVectorizer(ngram_range=(2, 2)).fit(bards_words)
print("어휘 사전 크기:", len(cv.vocabulary_))
print("어휘 사전:\n", cv.get_feature_names())

어휘 사전 크기: 14
어휘 사전:
 ['be fool', 'but the', 'doth think', 'fool doth', 'he is', 'himself to', 'is wise', 'knows himself', 'man knows', 'the fool', 'the wise', 'think he', 'to be', 'wise man']


연속된 토큰의 수가 커지면 보통 특성이 더 구체적이고 많이 만들어짐
- 예시에는 공통된 바이그램이 없다.

In [34]:
print("변환된 데이터 (밀집 배열):\n", cv.transform(bards_words).toarray())

변환된 데이터 (밀집 배열):
 [[0 0 1 1 1 0 1 0 0 1 0 1 0 0]
 [1 1 0 0 0 1 0 1 1 0 1 0 1 1]]


In [35]:
cv = CountVectorizer(ngram_range=(1, 3)).fit(bards_words)
print("어휘 사전 크기:", len(cv.vocabulary_))
print("어휘 사전:\n", cv.get_feature_names())

어휘 사전 크기: 39
어휘 사전:
 ['be', 'be fool', 'but', 'but the', 'but the wise', 'doth', 'doth think', 'doth think he', 'fool', 'fool doth', 'fool doth think', 'he', 'he is', 'he is wise', 'himself', 'himself to', 'himself to be', 'is', 'is wise', 'knows', 'knows himself', 'knows himself to', 'man', 'man knows', 'man knows himself', 'the', 'the fool', 'the fool doth', 'the wise', 'the wise man', 'think', 'think he', 'think he is', 'to', 'to be', 'to be fool', 'wise', 'wise man', 'wise man knows']


In [ ]:
# 시간이 오래 걸려서 테스트하지는 못 함

pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
              "tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3)]}

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print("최상의 크로스 밸리데이션 점수: {:.2f}".format(grid.best_score_))
print("최적의 매개변수:\n", grid.best_params_)

**이 다음 부터는 책 참조**

### 7.8 고급 토큰화, 어간 추출, 표제어 추출

이런 경우를 생각해보자
- drawback, drawbacks
- drawer, drawers
- drawing, drawings

BOW 모델에서는 drawback, drawbacks의 의미는 매우 가까워서 이를 구분하면 과대적합되기 쉽고, 모델이 훈련 데이터를 완전하게 활용하지 못함

명사에 단수, 복수가 있는 것처럼 여러 동사형과 관련 단어들을 다른 토큰으로 다루면 모델을 일반화하는 데 도움이 되지 않음


> 이 문제를 해결하려면 각 단어를 그 단어의 어간(stem)으로 표현해서 가튼 어간을 가진 모든 단어를 구분해야(또는 합쳐야) 합니다.
- 어간 : 활용어가 활용할 때에 변하지 않는 부분을 말함
- 보다, 보니, 보고 에서 보- // 먹다, 먹니, 먹고 에서 먹-
- 일일이 어미를 찾아 제외하는 규칙 기반 방식을 어간 추출(stemming)이라고 합니다.

> 대신 알려진 단어의 형태 사전(명시적이고 사람이 구축한 시스템)을 사용하고 문장에서 단어의 역할을 고려하는 처리 방식을 표제어 추출(lemmatization)이라고 하며 단어의 표준 형태를 표제어라고 한다.
- 표제어 추출 : 형태소 분석과 혼용해서 사용함.

> 표제어 추출, 어간 추출은 단어의 일반 형태를 추출하는 정규화(nomalization)의 형태로 볼 수 있다.

In [37]:
# !pip install nltk spacy
# python -m spacy download en

In [41]:
import spacy
import nltk

In [42]:
#spacy 영어 모델 로드

en_nlp = spacy.load('en')

#nltk의 PorterStemmer 객체 만들기

stemmer = nltk.stem.PorterStemmer()

In [43]:
def compare_normalization(doc):
    #spacy로 문서를 토큰화
    doc_spacy = en_nlp(doc)
    
    # spacy로 찾은 표제어를 출력
    print("표제어:")
    print([token.lemma_ for token in doc_spacy])
    
    # PorterStemmer로 찾은 토큰을 출력
    print("어간:")
    print([stemmer.stem(token.norm_.lower()) for token in doc_spacy])

In [44]:
compare_normalization(u"Our meeting today war worse than yesterday, " "I'm scared of meeting the clients tomorrow.")

표제어:
['-PRON-', 'meeting', 'today', 'war', 'bad', 'than', 'yesterday', ',', '-PRON-', 'be', 'scared', 'of', 'meet', 'the', 'client', 'tomorrow', '.']
어간:
['our', 'meet', 'today', 'war', 'wors', 'than', 'yesterday', ',', 'i', 'am', 'scare', 'of', 'meet', 'the', 'client', 'tomorrow', '.']


- 일반적으로 표제어 추출은 어간 추출보다 훨씬 복잡한 처리를 거침
- 하지만 머신러닝을 위해 토큰 정규화를 할 때는 어간 추출보다 좋은 결과를 낸다고 알려짐
- sklearn에서는 두 정규화 방법이 구현되어 있지 않지만, CountVectorizer에서 tokenizer 매개변수를 사용하여 문서를 토큰화하는 방법을 따로 지정 가능

In [46]:
# # 요구사항 : CountVectorizer의 정규표현식 기반 토큰 분할기를 사용하고 spacy에서 표제어 추출 기능만 이용
# # 이렇게 하려고 en_nlp.tokenizer(spacy 토큰 분할기)를 정규표현식 기반의 토큰 분할기로 바꿈

# import re
# regexp = re.compile('(?u)\\b\\w\\w+\\b')

In [48]:
# # spacy의 언어 모델을 로드하고 원본 토큰 분할기를 저장

# en_nlp = spacy.load('en', disable=['parser', 'ner'])
# old_tokenizer = en_nlp.tokenizer

**여기부터는 생략**

#### 7.8.1 (한국어판 부록) KoNLPy를 사용한 영화 리뷰 분석
- 별도 코드 작성 예정
- dataset : https://github.com/e9t/nsmc

### 7.9 토픽 모델링과 문서 군집화

텍스트 데이터에 자주 적용하는 특별한 기법으로 topic modeling이 있다.
- 이 용어는 비지도 학습으로 문서를 하나 또는 그 이상의 토픽으로 할당하는 작업을 통칭
- '정치', '스포츠', '금융' 등의 토피그로 묶을 수 있는 뉴스 데이터가 좋은 예
- 한 문서가 하나의 토픽에 할당되면 이것은 3장에서 본 것과 같은 문서를 군집시키는 문제가 됨
- 문서가 둘 이상의 토픽을 가질 수 있다면 이는 3장에서 본 분해 방법과 관련이 있다.
- 학습된 각 성분은 하나의 토픽에 해당하며 문서를 표현한 성분의 계수는 문서가 어떤 토픽에 얼마만큼 연관되어 있는지를 말해줌
- Latent Dirichlet Allocation = LDA = 잠재 디리클레 할당

#### 7.9.1 LDA

직관적으로 생각하면 LDA 모델은 함께 자주 나타나는 단어의 그룹 - 토픽을 찾는 것
- 또 LDA는 각 문서에 토픽의 일부가 혼합되어 있다고 간주
- 머신러닝에서 토픽은 우리가 일상 대화에서 말하는 '주제'가 아니고, 의미가 있든 없든 PCA나 NMF로 추출한 성분에 가까운 것
- LDA의 토픽에 의미가 있다고 하더라도, 이것은 우리가 주제라고 부르는 그것은 아님

In [50]:
# 최소 15%의 문서에서 나타나는 단어를 삭제한 후 가장 많이 등장하는 단어 10,000개에 대해 BOW 모델 생성

vect = CountVectorizer(max_features=10000, max_df=.15)
X = vect.fit_transform(text_train)

한 눈에 보기 쉬벡 10개의 토픽으로 토픽 모델 학습
- NMF의 성분과 비슷하게 토픽은 어떤 순서를 가지고 있지 않으며, 토픽의 수를 바꾸면 모든 토픽이 바뀌게 됨
- 기본 학습 방법("online") 대신 조금 느리지만 성능이 더 나은 "batch" 방법을 사용하고 모델 성능으 루이해 "max_iter"값을 증가

In [52]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, learning_method='batch', max_iter=25, random_state=0)

# 모델 생성과 변환을 한 번에 합니다.
# 변환 시간이 좀 걸리므로 시간을 절약하기 위해 동시에 처리

document_topics = lda.fit_transform(X)

3장에서 본 분해 방법처럼 LatentDirichletAllocation에는 토픽마다 각 단어의 중요도를 저장한 components_ 속성이 있다.
- components_의 크기는 (n_topics, n_words)이다.

In [ ]:
print("lda.components_.shape : ", lda.components_.shape)

각 토픽이 의미하는 것이 무엇인지 이해하기 위해 토픽에서 가장 중요한 단어를 확인

In [ ]:
# 토픽마다(components_의 행) 특성을 오름차순으로 정렬
# 내림차순이 되도록 [:, ::-1] 사용해 행릐 정렬을 반대로 바꿈

sorting = np.argsort(lda.components_, axis=-1)[:, ::-1]

# CountVectorizer 객체에서 특성 이름을 구하기

feature_names = np.array(vect.get_feature_names())

In [ ]:
mglearn.tools.print_topics(topics=topics, feature_names=feature_names,
                           sorting=sorting, topics_per_chunk=5, n_words=20)

- 너무 오래 걸려서 코드만 작성해두고 실행하지는 않았다.

### 7.10 잘 하자!